Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq

In [2]:

action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [4]:
# final_goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

def manhattan_dist(state):
    manhattan_dist = 0
    for row in range(len(state)):
        for col in range(len(state[row])):
            desired_value = row*len(state)+col+1
            if desired_value == len(state)**2: # Last element should be zero
                desired_value = 0
            # print(f"Coordinates: ({row},{col}) | Target: {final_goal[row][col]} | Value: {state[row][col]}".format())
            state_x, state_y = np.where(state==desired_value)
            manhattan_dist += np.abs(state_x - row) + np.abs(state_y - col)
    return manhattan_dist

# print(f"Manhattan Distance: {manhattan_dist(state)}".format()) 
# print("final_goal: \n", final_goal)
# print("state: \n", state)

In [5]:
class PuzzleNode:
    def __init__(self, state, g_cost=0, h_cost=0):
        self.state = state
        self.g_cost = g_cost
        self.h_cost = h_cost
        self.f_cost = g_cost + h_cost

    def __lt__(self, other):
        """Define less-than for comparison based on f_cost."""
        return self.f_cost < other.f_cost

    def __eq__(self, other):
        """Check equality based on the state, allowing comparison with np.array."""
        if isinstance(other, np.ndarray):
            return np.all(self.state == other)
        elif isinstance(other, PuzzleNode):
            return self.state == other.state
        return False  # Handle comparison with unsupported types

    def __hash__(self):
        """Define a hash function for using the object in sets or as dict keys."""
        return hash(self.state)

    def __repr__(self):
        """Readable representation of the node."""
        return (f"PuzzleNode(state=\n{self.state}, \ng_cost={self.g_cost}, "
                f"h_cost={self.h_cost}, f_cost={self.f_cost})")
        

In [6]:
def Astar(state):
    non_visited = []
    visited = []
    init_node = PuzzleNode(state, 0, manhattan_dist(state))
    heapq.heappush(non_visited,init_node)
    while True:
        curr_min_node = heapq.heappop(non_visited)
        possible_actions = available_actions(curr_min_node.state)
        heapq.heappush(visited,curr_min_node)

        for act_ind in range(len(possible_actions)):
            new_state = do_action(curr_min_node.state, possible_actions[act_ind])

            if new_state not in non_visited:
                new_node = PuzzleNode(new_state, curr_min_node.g_cost+1, manhattan_dist(new_state))
                heapq.heappush(non_visited,new_node)

                if manhattan_dist(new_state) == 0:
                    return new_node, visited, non_visited

        


    

In [7]:
PUZZLE_DIM = 3
RANDOMIZE_STEPS = 50
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/50 [00:00<?, ?it/s]

array([[1, 4, 2],
       [7, 0, 3],
       [5, 8, 6]])

In [8]:
last_node, visited, non_visited = Astar(state)


In [9]:
print("Last node: ", last_node)
print("Cost (Total number of actions evaluated): ", len(visited))
print("Quality (Number of actions in the solution): ", last_node.g_cost)
print("Quality + Cost: ", str(last_node.g_cost + len(visited)))

Last node:  PuzzleNode(state=
[[1 2 3]
 [4 5 6]
 [7 8 0]], 
g_cost=10, h_cost=[0], f_cost=[10])
Cost (Total number of actions evaluated):  26
Quality (Number of actions in the solution):  10
Quality + Cost:  36


In [10]:
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 50
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/50 [00:00<?, ?it/s]

array([[ 2,  5,  8,  3],
       [ 1, 11,  6,  4],
       [10, 13,  0, 12],
       [ 9, 14,  7, 15]])

In [11]:
last_node, visited, non_visited = Astar(state)

In [12]:
print("Last node: ", last_node)
print("Cost (Total number of actions evaluated): ", len(visited))
print("Quality (Number of actions in the solution): ", last_node.g_cost)
print("Quality + Cost: ", str(last_node.g_cost + len(visited)))

Last node:  PuzzleNode(state=
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]], 
g_cost=20, h_cost=[0], f_cost=[20])
Cost (Total number of actions evaluated):  362
Quality (Number of actions in the solution):  20
Quality + Cost:  382


In [13]:
PUZZLE_DIM = 5
RANDOMIZE_STEPS = 50
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/50 [00:00<?, ?it/s]

array([[ 1,  2,  3,  4,  5],
       [ 6, 12,  7,  9, 10],
       [11,  8, 14, 17, 15],
       [16, 22, 13,  0, 19],
       [21, 23, 24, 18, 20]])

In [14]:
last_node, visited, non_visited = Astar(state)

In [15]:
print("Last node: ", last_node)
print("Cost (Total number of actions evaluated): ", len(visited))
print("Quality (Number of actions in the solution): ", last_node.g_cost)
print("Quality + Cost: ", str(last_node.g_cost + len(visited)))

Last node:  PuzzleNode(state=
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24  0]], 
g_cost=16, h_cost=[0], f_cost=[16])
Cost (Total number of actions evaluated):  190
Quality (Number of actions in the solution):  16
Quality + Cost:  206
